In [ ]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

from xradio.data.datasets import download

# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

## Download dataset

In [1]:
from xradio.data.datasets import download

ms_file = "global_vlbi_gg084b_reduced.ms"
download(file=ms_file, source="dropbox")

url https://www.dropbox.com/scl/fi/we8898t15tfz2eogenhvb/global_vlbi_gg084b_reduced.ms.zip?rlkey=mtp82hozzlvl92fmizu7fi577&dl


./global_vlbi_gg084b_reduced.ms.zip: 100%|██████████| 15.0M/15.0M [00:00<00:00, 35.7MiB/s]


## Convert to Processing Set

In [2]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

partition_scheme = "ddi_intent_field"
# partition_scheme="ddi_state"

main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "global_vlbi_gg084b_reduced.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    partition_scheme=partition_scheme,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

## Processing Set

In [3]:
ps_name = outfile

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_name=outfile)
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency
0,global_vlbi_gg084b_reduced_ddi_1_intent_None_f...,1,None,1,EM170817,4.916000e+09,4.931500e+09
1,global_vlbi_gg084b_reduced_ddi_1_intent_None_f...,1,None,0,J1311-2329,4.916000e+09,4.931500e+09
2,global_vlbi_gg084b_reduced_ddi_0_intent_None_f...,0,None,0,J1311-2329,4.900500e+09,4.916000e+09
3,global_vlbi_gg084b_reduced_ddi_0_intent_None_f...,0,None,1,EM170817,4.900500e+09,4.916000e+09


In [4]:
ps.keys()

dict_keys(['global_vlbi_gg084b_reduced_ddi_1_intent_None_field_id_1', 'global_vlbi_gg084b_reduced_ddi_1_intent_None_field_id_0', 'global_vlbi_gg084b_reduced_ddi_0_intent_None_field_id_0', 'global_vlbi_gg084b_reduced_ddi_0_intent_None_field_id_1'])

In [5]:
ps['global_vlbi_gg084b_reduced_ddi_1_intent_None_field_id_1']

<xarray.Dataset>
Dimensions:                     (time: 150, baseline_id: 28, frequency: 32,
                                 polarization: 4, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int64 dask.array<chunksize=(28,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int64 dask.array<chunksize=(28,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 0 1 2 3 4 ... 23 24 25 26 27
  * frequency                   (frequency) float64 4.916e+09 ... 4.932e+09
  * polarization                (polarization) <U2 'RR' 'RL' 'LR' 'LL'
  * time                        (time) float64 1.527e+09 1.527e+09 ... 1.527e+09
  * uvw_label                   (uvw_label) <U1 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 dask.array<chunksize=(20, 28), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool dask.array<chunksize=(20, 28, 1, 4), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 dask.array<chunksize=(20, 28), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 dask.array<chunksize=(20, 28, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 dask.array<chunksize=(20, 28, 1, 4), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 dask.array<chunksize=(20, 28, 1, 4), meta=np.ndarray>
Attributes:
    data_groups:  {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'VISI...
    ddi:          1
    field_info:   {'code': '', 'delay_direction': {'attrs': {'frame': 'FK5', ...
    intent:       None
    antenna_xds:  <xarray.Dataset>\nDimensions:        (antenna_id: 23, xyz_l...

In [ ]:
ps['global_vlbi_gg084b_reduced_ddi_1_intent_None_field_id_1'].attrs['antenna_xds']